In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import requests
import pandas as pd
import time
import os

# --- Configuration ---
# API Endpoint for FDIC Bank Branch Locations, confirmed via FDIC BankFind Suite API Docs (https://api.fdic.gov/banks/docs/)
BASE_URL = "https://banks.data.fdic.gov/api/locations"
# Define the city and state to filter by.
TARGET_STATE_ABBR = "CA"
TARGET_CITY_NAME = "LOS ANGELES" # Changed to target city name
# TARGET_FIPS_CODE = "06037" # FIPS code for Los Angeles County (no longer used for filtering in process_and_save_data)

# Columns to extract from the API response
FIELDS_TO_EXTRACT = [
    "ID", "NAME", "CITY", "STALP", "COUNTY", "ZIP",
    "ADDRESS", "DEPSUM", "ESTYMD", "UNINUM", "OFFNAME",
    "OFFGEO", "FIPS" # FIPS code is kept for data integrity, but filtering is now by City
]

def fetch_fdic_locations_for_state(state_abbr):
    """
    Fetches all bank branch location data for a specific state
    from the FDIC BankFind API. City filtering will be done post-fetch.
    """
    print(f"Starting data extraction for {state_abbr} (all locations in state)...")

    # Construct the filter string to fetch all branches in the target state
    filter_string = f'STALP:"{state_abbr}"'

    # Base parameters for the API request
    params = {
        'filters': filter_string,
        'fields': ",".join(FIELDS_TO_EXTRACT),
        'limit': 10000,  # Max limit per request
        'offset': 0,
        'sort_by': 'ID',
        'format': 'json'
    }

    all_data = []
    total_results = 0
    page_count = 0

    # Loop to handle API pagination
    while True:
        page_count += 1
        print(f"Fetching page {page_count} (Offset: {params['offset']})... (State: {state_abbr})")

        try:
            # Make the API request
            response = requests.get(BASE_URL, params=params, timeout=30)
            response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
            data = response.json()

            # Check for data presence
            if not data.get('data'):
                print("End of data reached or empty response received for state.")
                break

            # Append current page data to the master list
            all_data.extend(data['data'])

            # Extract metadata for pagination
            meta = data.get('meta', {})
            total_results = meta.get('total', 0)
            next_offset = meta.get('next_offset')

            # Break if no next offset or we have fetched all expected records
            if next_offset is None or len(all_data) >= total_results:
                break

            # Update offset for the next request
            params['offset'] = next_offset

            # Pause to prevent rate-limiting
            time.sleep(1)

        except requests.exceptions.RequestException as e:
            print(f"An error occurred during API request: {e}")
            print("Stopping extraction.")
            break

    print(f"\nExtraction complete for state {state_abbr}. Total records fetched: {len(all_data)} (Total available: {total_results})")
    return all_data

def process_and_save_data(data, target_city_name):
    """
    Converts the list of bank data into a DataFrame,
    filters by City name, and saves it to CSV.
    """
    if not data:
        print("No data to process. Output file will not be created.")
        return

    # Extract the 'data' field from each item and convert to a DataFrame
    df = pd.DataFrame([item['data'] for item in data if item.get('data')], columns=FIELDS_TO_EXTRACT)

    if df.empty:
        print("DataFrame is empty after initial processing.")
        return

    # Rename columns for clarity
    column_mapping = {
        "ID": "FDIC_ID",
        "NAME": "Institution_Name",
        "CITY": "City",
        "STALP": "State",
        "COUNTY": "County_Name",
        "ZIP": "ZIP_Code",
        "ADDRESS": "Address",
        "DEPSUM": "Total_Deposits_Millions",
        "ESTYMD": "Est_Year",
        "UNINUM": "Branch_ID",
        "OFFNAME": "Branch_Name",
        "OFFGEO": "Geographic_Coordinates",
        "FIPS": "County_FIPS_Code"
    }
    df = df.rename(columns=column_mapping)

    # Filter the DataFrame by the target City name (case-insensitive)
    initial_record_count = len(df)
    df = df[df['City'].str.upper() == target_city_name.upper()].copy() # .copy() to avoid SettingWithCopyWarning
    print(f"Filtered data from {initial_record_count} records to {len(df)} for City '{target_city_name}'.")

    if df.empty:
        print(f"No records found for City '{target_city_name}'. Output file will not be created.")
        return

    # Clean up and reorder columns for a nice output CSV
    final_columns = [
        "Institution_Name", "Branch_Name", "Address", "City", "State",
        "ZIP_Code", "County_Name", "County_FIPS_Code", "Branch_ID", "FDIC_ID",
        "Total_Deposits_Millions", "Est_Year", "Geographic_Coordinates"
    ]
    df = df[final_columns]

    # Define output file path
    output_filename = f"la_city_ca_fdic_branches.csv"

    # Save to CSV
    df.to_csv(output_filename, index=False)

    print(f"\nSuccessfully saved {len(df)} records to: {os.path.abspath(output_filename)}")
    print("\n--- First 5 Rows of Data ---")
    print(df.head().to_markdown(index=False, numalign="left", stralign="left"))


if __name__ == "__main__":
    # Call the main function to fetch the data for the entire state
    # The function name has been updated to reflect fetching for state, not county.
    locations_data = fetch_fdic_locations_for_state(TARGET_STATE_ABBR)

    # Process and save the data, including client-side filtering by City name
    process_and_save_data(locations_data, TARGET_CITY_NAME)

Starting data extraction for CA (all locations in state)...
Fetching page 1 (Offset: 0)... (State: CA)

Extraction complete for state CA. Total records fetched: 5584 (Total available: 5584)
Filtered data from 5584 records to 298 for City 'LOS ANGELES'.

Successfully saved 298 records to: /content/la_city_ca_fdic_branches.csv

--- First 5 Rows of Data ---
| Institution_Name              | Branch_Name             | Address                                 | City        | State   | ZIP_Code   | County_Name   | County_FIPS_Code   | Branch_ID   | FDIC_ID   | Total_Deposits_Millions   | Est_Year   | Geographic_Coordinates   |
|:------------------------------|:------------------------|:----------------------------------------|:------------|:--------|:-----------|:--------------|:-------------------|:------------|:----------|:--------------------------|:-----------|:-------------------------|
| City National Bank            | City National Bank      | 555 S Flower St                         | L